In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds
imdb,info=tfds.load('imdb_reviews',with_info=True,as_supervised=True)

In [ ]:
import numpy as np
train_data,test_data=imdb['train'],imdb['test']
train_sentences=[]
train_labels=[]
test_sentences=[]
test_labels=[]
for s,l in train_data:
    train_sentences.append(str(s.numpy()))
    train_labels.append(l.numpy())
for s,l in test_data:
    test_sentences.append(str(s.numpy()))
    test_labels.append(l.numpy())

train_labels=np.array(train_labels)
test_labels=np.array(test_labels)

In [ ]:
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='post'
oov_tok='<OOV>'
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(train_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)
testing_sequences=tokenizer.texts_to_sequences(test_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs=10
model.fit(padded,train_labels,epochs=num_epochs,validation_data=(testing_padded,test_labels))

In [ ]:
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])


In [ ]:
import io

# Get the weights of the embedding layer
weights = model.layers[0].get_weights()[0]  # Assuming the embedding layer is the first layer in your model

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]  # Now 'weights' is defined
    out_m.write(word + '\n')
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()